In [ ]:
from __future__ import absolute_import, division

import os
import urllib, cStringIO

import pymongo as pm

import numpy as np
import scipy.stats as stats
import pandas as pd
import json
import re

from PIL import Image
import base64
import sys

import matplotlib
from matplotlib import pylab, mlab, pyplot
%matplotlib inline
from IPython.core.pylabtools import figsize, getfigs
plt = pyplot
import seaborn as sns
sns.set_context('talk')
sns.set_style('darkgrid')

from IPython.display import clear_output

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")

### Setup

In [ ]:
# set vars 
auth = pd.read_csv('auth.txt', header = None) # this auth.txt file contains the password for the sketchloop user
pswd = auth.values[0][0]
user = 'sketchloop'
host = 'rxdhawkins.me' ## cocolab ip address

# have to fix this to be able to analyze from local
import pymongo as pm
conn = pm.MongoClient('mongodb://sketchloop:' + pswd + '@127.0.0.1')
db = conn['kiddraw']
coll = db['tracing_eval']


### Preprocessing
#### Basic Information about mturk ratings

In [ ]:
#iteration_names = ['pilot0', 'pilot1']
iteration_names = ['pilot0']
for t in iteration_names:
    data_t = coll.find({'$and':[{'iterationName':t},{'workerId':{'$exists':True}}]}).sort('startTrialTime').sort('workerId')
    print 'Iteration {} has {} ratings'.format(t, data_t.count())
    

In [ ]:
all_sessions = coll.distinct('workerId') 
print "Currently, {} mturkers have finished the game".format(len(all_sessions)-1)

In [ ]:
expected = []
finished = []
repeated = 0
more_catch = 0

for s in all_sessions:
    if s == '':
        continue
    finished_trials = coll.find({ "workerId": s})
    default_trials = coll.find({"workerId": s,'category':'catch'})
    
    if finished_trials.count() == 105 and default_trials.count() == 5:
        expected.append(finished_trials)
    if finished_trials.count() == 105:
        finished.append(finished_trials)
    if finished_trials.count() >105:
        iterName = list(finished_trials)[0]['iterationName']
        print "Worker {} in iteration {} has finished {} trials. {} trials are catch trials".format(s, iterName, finished_trials.count(), default_trials.count())
        repeated += 1
    if default_trials.count() > 5:
        more_catch += 1

print "\nAmong {} mturkers, {} finished the survey. ".format(len(all_sessions)-1, len(finished))
print "Among {} mturkers who finished the game, {} was completed in a normal condition (105 trials, 5 catch)".format(len(finished), len(expected))
print "{} mturkers finished more that 105 trials.".format(repeated)
print "{} mturkers encountered more than 5 catch/default trials".format(more_catch)
    

#### Evaluating raters by the following criterions and exclude raters who gave random responses:
    1. 80% Answers of the catch trial are consistent and the average is larger than 2 （on a 0-4 scale)
    2. There are no more than 20 continuous trials that have the same rating

In [ ]:
try:
    all_sessions.remove('')
except:
    print " '' is already removed"
    
excluded_workerId = ['', ] # '' is 'no workerId'
valid_ratings = []

exc_incatch = []
exc_2catch = []
exc_20 = []

tolerance = 0.8
threshold = 2

for s in all_sessions: 
    strials = coll.find({'workerId':s})
    
    # if no ratings, exclude the current rater
    if strials.count() == 0:
        excluded_workerId.append(s)
        continue
    
    if list(strials)[0]['iterationName'] not in iteration_names:
        excluded_workerId.append(s)
        continue
        
    # if answers of catch trials are inconsistent, exclude the rater
    catch = coll.find({'workerId':s, 'category':'catch'})
    if catch.count() == 0: # no catch trials = no ratings at all. The first trial is the catch trial
        excluded_workerId.append(s)
        continue
        
    catch_answer = []
    for t in catch:
        catch_answer.append(int(t['button_pressed']))
    
    catch_answer = np.array(catch_answer)
    
    # check if all answers are larger than the threshold
    if len(np.where(catch_answer>threshold)[0]) != len(catch_answer): 
        excluded_workerId.append(s)
        exc_2catch.append(s)
        continue
    
    # check if tolerance% of answers are the same number
    counts = np.max(np.bincount(catch_answer))
    if counts < len(catch_answer) * tolerance:
        excluded_workerId.append(s)
        exc_incatch.append(s)
        continue
    
    # if more than 20 continuous trials are all the same, exclude the rater
    # 1. get all non-catch answers
    trials = coll.find({'workerId':s, 'category':{'$not':{'$eq':'catch'}}}).sort('trial_index')
    if trials.count() == 0:
        excluded_workerId.append(s)
        continue
    
    nc_answer = []
    current_rating = pd.DataFrame(list(trials))
    nc_answer = current_rating['button_pressed']
    
    # 2. check for repetitions
    rep = nc_answer[0]
    rep_times = 0

    for v in nc_answer[1:]:
        if v == rep:
            rep_times+= 1
            if rep_times == 20: 
                print 'catch'
                break
        else:
            rep = v
            rep_times = 0
    
    if rep_times == 20:
        excluded_workerId.append(s)
        exc_20.append(s)
        continue
            
    valid_ratings.append(current_rating)
    
print "Excluded workerIds are {}".format(excluded_workerId)
print "{} participants has inconsistent catch trial answers".format(len(exc_incatch))
print "{} participants gave a comparatively low rating (0, 1, 2) to one or more catch trials".format(len(exc_2catch))
print "{} participants has the same ratings for more than 20 continuous trials".format(len(exc_20))

In [ ]:
# write valid responses to a csv file
all_norm = pd.concat(valid_ratings)
all_norm.to_csv('valid_rating2.csv')

In [ ]:
filtered = [s for s in all_sessions if s not in excluded_workerId]
print 'We have {} participants in total. After the filtering process, there are {} participants remaining'.format(len(all_sessions), len(filtered))

num_ratings = coll.find({'workerId':{'$nin': excluded_workerId}, 'category':{'$not':{'$eq':'catch'}}})
print "We have {} ratings from {} valid raters on {} children's tracings.".format(num_ratings.count(), len(filtered), len(num_ratings.distinct('session_id')))

unique_tracings = num_ratings.distinct('session_id')
print " {} mturkers has rated {} tracings".format(len(filtered), len(unique_tracings))

### Combine Human Rating Results with Image Registration Model Results
For all tracings in the museumstation, the csv file $model$_$result$ contains model outputs. The model output consists of shape error and spatial error for each tracing image.

In [ ]:
# read relevant files
model_result = 'museumstation_tracing_ncc.csv'
rating_result = 'valid_rating2.csv'
data = pd.read_csv(model_result)
all_norm = pd.read_csv(rating_result)
all_norm = all_norm.drop(all_norm[all_norm['iterationName'] == 'testing2'].index)
all_norm = all_norm.drop(all_norm[all_norm['session_id'].str.contains('CDM_photodraw')].index)

In [ ]:
# Process the model result
# 1. normalize rotation, translation, and scaling
data['norm_r'] = (data['rotate'] - data['rotate'].mean())/data['rotate'].std()
data['norm_t'] = (data['translate'] - data['translate'].mean())/data['translate'].std()
data['norm_s'] = (data['scale'] - data['scale'].mean())/data['scale'].std()

# 2. summarize the three varaibles
w_r, w_t, w_s = 1, 1, 1
data['spatial'] = data['norm_r'] * w_r + data['norm_t'] * w_t + data['norm_s'] * w_s

try:
    data = data.drop(["Unnamed: 0", "Unnamed: 0.1"], axis=1)
except:
    print "Already dropped"

data.to_csv(model_result)

In [ ]:
norating = data.loc[data['human_norm_rating'].isnull()].index
data.loc[norating, 'human_norm_rating'] = -10.0
data.loc[norating, 'human_rating'] = -10.0

In [ ]:
data['button_pressed'] = -10
new_data = []
drop_index = []

for di, d in data.iterrows():
#     if di>50:
#         break
    human_rating = all_norm[(all_norm['session_id'] == d['session_id']) & (all_norm['category'] == d['category'])]
    if human_rating.shape[0]!=0: # if the current item is rated and there exists at least one rating
        # duplicate the same row for several times
        dnew = pd.DataFrame(np.tile(d.values, (human_rating.shape[0],1)))
        dnew.columns = data.columns
        ratings = list(human_rating['button_pressed'].astype(int))
        
        # fill in the 'button_pressed' value
        for hi, h in dnew.iterrows():
            dnew.loc[hi, 'button_pressed'] = ratings[hi]
        
        new_data.append(dnew)
        drop_index.append(di)
        all_norm = all_norm.drop(human_rating.index)

print len(drop_index)
data = data.drop(drop_index)
new_data.append(data)
new_data = pd.concat(new_data)

try:
    new_data = new_data.drop('Unnamed: 0', axis=1)
except:
    print 'already dropped'  

new_data = new_data.reset_index()
try:
    new_data = new_data.drop('index', axis=1)
except:
    print 'already dropped'
    
new_data.to_csv('tracing_ordinal_data.csv')   

In [ ]:
len(pd.unique(data[['session_id', 'category']].values.ravel()))

In [ ]:
len(pd.unique(new_data[['session_id','category']].values.ravel()))

In [ ]:
data.drop_duplicates(subset=['session_id', 'category'], keep="last")
data.drop('Unnamed: 0', axis=1)